In [3]:
using Random
using LinearAlgebra
using Statistics
using StatsBase
using Plots
using Base.Threads
using Flux
using Base.Iterators: product
using Flux: Optimise
using Flux: ADAM, params, update!
using Serialization

In [22]:
include("permanents.jl")

#n_sessions = 2000  # Number of new sessions per iteration
#learning_rate = 0.001  # Learning rate, increase this to converge faster
percentile = 90  # Top 100-x percentile the agent will learn from
super_percentile = 90  # Top 100-x percentile of that survives to the next generation

90

# Helper functions

In [5]:
function board_to_string(board::Vector{Int64}, n)
    # board is currently a Vector
    board = reshape(convert(Vector{Int}, board), (n,n))
    
    output = "["
    for i in 1:n-1
        output = output * string(board[:,i]) * '\n' * ' '
    end
    output = output * string(board[:,n]) * "]"

    return output
end

board_to_string (generic function with 1 method)

In [6]:
function new_point_allowed(one_indices, new_point_index, n)
    row = ceil(Int, new_point_index / n)
    col = new_point_index % n
    if col == 0; col = n end # julia is 1-indexed, so x*n mod(n) must be n rather than 0

    point_allowed = true

    for i in 1:length(one_indices) 
        for j in i+1:length(one_indices)
            point_one_row = ceil(Int, one_indices[i] / n)
            point_two_row = ceil(Int, one_indices[j] / n)
            #if point_one_row == 0; point_one_row end
            #if point_two_row == 0; point_two_row end

            point_one_col = one_indices[i] % n
            point_two_col = one_indices[j] % n
            if point_one_col == 0; point_one_col = n end
            if point_two_col == 0; point_two_col = n end

            if (  (row == point_one_row)
                | (row == point_two_row)
                | (point_one_row == point_two_row)
                | (col == point_one_col)
                | (col == point_two_col)
                | (point_one_col == point_two_col))
                continue
            end
            
            # ensure point_one_col < point_two_col
            if (point_two_col < point_one_col)
                point_one_row, point_two_row = point_two_row, point_one_row
                point_one_col, point_two_col = point_two_col, point_one_col
            end

            # new point as 1 in a valid 312-pattern
            if ((point_two_row < point_one_row < row) & (point_one_col < col < point_two_col))
                point_allowed = false
                break
            end

            # new point as 2 in a valid 312-pattern
            if ((point_two_row < row < point_one_row) & (col < point_one_col))
                point_allowed = false
                break
            end

            # new point as 3 in a valid 312-pattern
            if ((row < point_one_row < point_two_row) & (point_two_col < col))
                point_allowed = false
                break
            end
        end

        if point_allowed == false
            break
        end
        
    end

    return point_allowed
end

new_point_allowed (generic function with 1 method)

# Add points

In [7]:
# helper function to add 3 of the 5 free points present in all 312-avoiding matrices
function add_free_points(input_state, n, step)
    action_taken = 0 #zeros(n^2)
    cur_state = copy(input_state)

    if step == 1
        action_index = 1
    elseif step == 2
        action_index = n^2 - n + 1
    else # step == 3
        action_index = n^2
    end

    cur_state[action_index] = 1
    action_taken = action_index

    return cur_state, action_taken
end

add_free_points (generic function with 1 method)

In [8]:
function add_corner(input_state, action_vec, n, row_boundary, col_boundary)
    point_added = false
    action_taken = 0 #zeros(n^2)
    action_row = 0
    action_col = 0
    cur_state = copy(input_state)

    terminal = false

    ###
    # 1-indexing will make complicated, can just shift to 0-index then add later?
    # probably not that bad to use 1-indexing
    while !point_added
        action_index = StatsBase.sample(collect(1:length(action_vec)), Weights(action_vec), 1)[1]
        action_row = ceil(Int, action_index / n)
        action_col = action_index % n
        if action_col == 0; action_col = n end
        
        if (  ((action_row == n-1) & (action_col != n)) # the only corner in row n-1 possible is at (n-1, n)
            | ((action_col == 2) & (action_row != 1)) # only possible corner in col 2 is in row 1
            | ((row_boundary == 1) & (action_row != 1)) # ensures that first corner is in row 1.
            )
            action_vec[action_index] = 0
            action_vec = action_vec ./ sum(action_vec)
        elseif ((row_boundary <= action_row < n) & (col_boundary <= action_col)) # no corners in row n
            cur_state[action_index] = 1
            #action_taken[action_index] = 1
            action_taken = action_index
            point_added = true
        else
            action_vec[action_index] = 0
            action_vec = action_vec ./ sum(action_vec)
        end
    end

    if action_col == n
        terminal = true
    end

    return cur_state, action_taken, terminal, action_row, action_col
end

add_corner (generic function with 1 method)

In [9]:
function add_point(input_state, action_vec, n)
    point_added = false
    action_taken = 0 #zeros(Int, 1)
    cur_state = copy(input_state)
    
    while !point_added
        action_index = StatsBase.sample(collect(1:length(action_vec)), Weights(action_vec), 1)[1]
        one_indices = findall(!iszero, input_state)
        action_allowed = new_point_allowed(one_indices, action_index, n)
        
        if (cur_state[action_index] == 0) & (action_allowed)
            cur_state[action_index] = 1
            action_taken = action_index
            point_added = true
        else
            action_vec[action_index] = 0
            action_vec = action_vec ./ sum(action_vec)
        end
    end

    return cur_state, action_taken
end

add_point (generic function with 1 method)

# Generate sessions and super/ elite sessions

In [10]:
function generate_session(agent, corner_agent, n_sessions, n)
    states = zeros(Int, n_sessions, 4*n - 4 + 1, n^2)
    actions = zeros(Int, n_sessions, 4*n - 4 + 1)
    scores = zeros(Int, n_sessions)
    cur_state = zeros(Int, n^2)

    step = 0

    for i in 1:n_sessions # [1, n_sessions] inclusive
        step = 0
        #println(" ")
        #println("Sessions: " * string(i))
        # add first 3 free points, i.e., (1,1), (n,1), (n,n)
        while step < 3
            step += 1
            cur_state .= states[i, step, :]
            next_state, action = add_free_points(cur_state, n, step)

            actions[i, step] = action
            states[i, step + 1, :] = next_state
        end

        # add corners
        corner_num = 0
        corner_list = Vector{Int}(undef, 0)

        row_boundary, col_boundary = 1, 2
        #row_one_set = false
        terminal = false
        while !terminal
            step += 1
            cur_state .= states[i, step, :]

            output = corner_agent(cur_state)

            next_state, action, terminal, row_added, col_added = add_corner(cur_state, output, n, row_boundary, col_boundary)
            
            corner_index = (row_added - 1) * n + col_added
            push!(corner_list, corner_index)
            
            corner_num += 1

            row_boundary = row_added + 1
            col_boundary = col_added + 1

            actions[i, step] = action
            states[i, step + 1, :] = next_state
        end

        sort!(corner_list)

        # add induced corners
        for index in corner_list
            step += 1
            cur_state .= states[i, step, :]

            corner_row = ceil(Int, index / n)
            corner_col = index % n
            if corner_col == 0; corner_col = n end
            # induced_corner = (i+1, j-1) -> (i+1)n + (j-1), where i=corner_row and j=corner_col
            # so, e.g., induced_corner_row = corner_row + 1, induced_corner_col = corner_col - 1
            induced_corner_row, induced_corner_col = corner_row + 1, corner_col - 1

            # in general (for julia), index = (row - 1)*n + col
            induced_corner_index = (induced_corner_row - 1)*n + (induced_corner_col)
            cur_state[induced_corner_index] = 1

            actions[i, step] = induced_corner_index
            states[i, step + 1, :] = cur_state
        end

        ## add zig-zag
        push!(corner_list, n^2) # n^2 is not a corner, but we append it to work with while-loop below
        cur_path_element = 2 # first element on the zig-zag path, besides upper-left corner
        target_corner_num = 1
        target_corner = corner_list[target_corner_num]

        # first row of zig-zag path
        while cur_path_element != target_corner
            step += 1
            cur_state .= states[i, step, :]
            cur_state[cur_path_element] = 1

            #println("Step: " * string(step))
            #println("Curr element: " * string(cur_path_element))

            actions[i, step] = cur_path_element
            states[i, step + 1, :] = cur_state

            cur_path_element += 1 # step element by column, fixing row
        end

        #println("Length: " * string(length(corner_list)))
        #println(corner_list)
        # remaining points on zig-zag path
        target_corner_num += 1
        while target_corner_num <= length(corner_list)
            target_corner = corner_list[target_corner_num]
            target_row = ceil(target_corner / n)
            # target_col not needed

            cur_row = ceil(cur_path_element / n)
            #cur_col = cur_path_element % n
            #if cur_col == 0; cur_col = n end
            #println("Target corner num: " * string(target_corner_num))
            #println("Target corner: " * string(target_corner))
            while (cur_path_element != target_corner) & (cur_path_element != n^2 - n) # prevents incorrect row OR column
                if cur_row != target_row # must move row
                    # add point
                    step += 1
                    cur_path_element += n # step element by one row, fixing column
                    cur_state .= states[i, step, :]
                    cur_state[cur_path_element] = 1

                    #println("Step: " * string(step))
                    #println("Curr element: " * string(cur_path_element))
        
                    actions[i, step] = cur_path_element
                    states[i, step + 1, :] = cur_state
        
                    cur_row += 1
                else # cur_col != target_col, i.e., correct row but must move column
                    cur_path_element += 1 # step element by one column, fixing row
                    if cur_path_element != target_corner # move column
                        # add point
                        step += 1
                        cur_state .= states[i, step, :]
                        cur_state[cur_path_element] = 1

                        #println("Step: " * string(step))
                        #println("Curr element: " * string(cur_path_element))

                        actions[i, step] = cur_path_element
                        states[i, step + 1, :] = cur_state

                        #cur_col += 1
                    end
                end
            end
            target_corner_num += 1
            #println("target num after: " * string(target))
            #println("step after loop:" * string(step))
        end

        # add remaining points
        while step < 4*n - 4 # there are 4*n - 4 steps total, starts at 0 and ends at 4*n - 4 - 1
            step += 1
            # current board
            cur_state .= states[i, step, :]

            output = agent(cur_state)

            next_state, action = add_point(cur_state, output, n)

            actions[i, step] = action
            states[i, step + 1, :] = next_state
        end

        final_state = states[i, step + 1, :]
        final_state_matrix = reshape(final_state, (n,n))
        scores[i] = glynn(final_state_matrix)
    end

    return states, actions, scores
end

generate_session (generic function with 1 method)

In [11]:
function select_super_sessions(states_batch, actions_batch, rewards_batch, super_percentile)
    counter = n_sessions * (100 - super_percentile) / 100
    reward_threshold = quantile(rewards_batch, super_percentile / 100)

    super_states = zeros(Int, 0, size(states_batch, 2), size(states_batch, 3))
    super_actions = Matrix{Int}(undef, 0, size(actions_batch, 2))
    super_rewards = Vector{Int}(undef, 0)
    
    for i in 1:size(states_batch, 1)
        if rewards_batch[i] >= reward_threshold - 0.000001 && counter > 0
            temp_state = reshape(states_batch[i, :, :], (1,size(states_batch[i, :, :])...))
            super_states = cat(super_states, temp_state; dims=1)

            temp_actions = reshape(actions_batch[i, :], (1,size(actions_batch[i, :])...))
            super_actions = cat(super_actions, temp_actions; dims=1)

            push!(super_rewards, rewards_batch[i])

            counter -= 1
        end
    end

    return super_states, super_actions, super_rewards
end

select_super_sessions (generic function with 1 method)

In [12]:
function select_elites(states_batch, actions_batch, rewards_batch, percentile)
    counter = n_sessions * (100 - percentile) / 100
    reward_threshold = quantile(rewards_batch, percentile / 100)

    elite_states = Matrix{Int}(undef, 0, size(states_batch, 3))
    elite_actions = Vector{Int}(undef, 0)
    
    for i in 1:size(states_batch, 1)
        if rewards_batch[i] >= reward_threshold - 0.000001 && counter > 0
            for item in eachrow(states_batch[i, :, :])
                temp_state = reshape(item, 1, size(states_batch, 3)) # size of board
                elite_states = vcat(elite_states, temp_state)
            end

            for item in actions_batch[i, :]
                push!(elite_actions, item)
            end

            counter -= 1
        end
    end

    return elite_states, elite_actions
end

select_elites (generic function with 1 method)

# Training function

In [31]:
function train(board_size, filename)
    n_sessions = 2000
    learning_rate = 0.001
    corner_learning_rate = 0.0001

    n = board_size
    input_space = n*n

    first_layer_neurons = 128
    second_layer_neurons = 64
    third_layers_neurons = 4

    # Define the neural network architecture (similar to PyTorch)
    net = Chain(
        Dense(n^2, first_layer_neurons, relu),
        Dense(first_layer_neurons, second_layer_neurons, relu),
        Dense(second_layer_neurons, third_layers_neurons, relu),
        Dense(third_layers_neurons, n^2, σ)
    )

    # Define the neural network architecture (similar to PyTorch)
    corner_net = Chain(
        Dense(n^2, first_layer_neurons, relu),
        Dense(first_layer_neurons, second_layer_neurons, relu),
        Dense(second_layer_neurons, third_layers_neurons, relu),
        Dense(third_layers_neurons, n^2, σ)
    )

    # Defining the loss function and optimizer (similar to PyTorch)
    criterion(y_pred, y_true) = Flux.binarycrossentropy(y_pred, y_true)
    optimizer = Optimise.ADAM(learning_rate)
    corner_optimizer = Optimise.ADAM(corner_learning_rate)

    # Global lists
    global super_states = Array{Int}(undef, 0, 4*n - 4 + 1, n^2)
    global super_actions = Array{Int}(undef, 0, 4*n - 4 + 1)
    global super_rewards = Int[]

    cur_best_reward = 0
    cur_best_board = []
    cur_best_game = []
    local best_states_set
    
    for i in 1:20000
        println("\n GENERATION $i")
        states_batch, actions_batch, rewards_batch = generate_session(net, corner_net, n_sessions, n)
        #println("new batch size: " * string(size(states_batch)))

        if i > 1
            states_batch = cat(states_batch, super_states; dims=1)
            actions_batch = cat(actions_batch, super_actions; dims=1)
            rewards_batch = cat(rewards_batch, super_rewards; dims=1)
            #println("super size: " * string(size(super_states)))
        end
        #println("batch size after cat: " * string(size(states_batch)))

        elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile)

        ###
        # sessions[1][i,:,:] are the states corresponding to the ith session
        # likewise for actions and rewards
        # the outer dimension is now the number of sessions
        # the next dimension indicates (1 = states, 2 = actions, 3 = rewards)
        # reverse sort the sessions (in the outer dimension) based on the rewards
        # sessions = select_super_sessions(states_batch, actions_batch, rewards_batch, super_percentile)
        ###
        super_states, super_actions, super_rewards = select_super_sessions(states_batch, actions_batch, rewards_batch, super_percentile)
        sessions = [super_states, super_actions, super_rewards]

        ### error: this is not updating super_states, below we concat with old super_states.. error below
        #println("new super size: " * string(size(super_states))) # this is not new, this is old
        #println(size(sessions[1]))
        super_sessions = [[sessions[1][i,:,:], sessions[2][i,:], sessions[3][i]] for i in 1:length(sessions[3])]
        #println(size(super_sessions))
        sort!(super_sessions, by = x -> x[3], rev=true)

        #### delete
        #if i == 2
        #    println(board_to_string(super_sessions[1][1][4*n - 4 + 1,:],n)) # 1st session, states of first session, last state
        #    println(super_sessions[1][3])
        #end

        # optimize
        # Backward pass (gradients calculation) and optimization (similar to PyTorch)
        net_outputs = zeros(Float32, size(elite_states, 1), 1)
        corner_net_outputs = zeros(Float32, size(elite_states, 1), 1)
        for i in 1:size(elite_states,1)
            net_outputs[i] = net(elite_states[i, :, :])[1]
            corner_net_outputs[i] = corner_net(elite_states[i, :, :])[1]
        end

        loss = criterion(net_outputs, elite_actions)
        grads = gradient(() -> loss, params(net))
        Optimise.update!(optimizer, params(net), grads)

        loss = criterion(corner_net_outputs, elite_actions)
        grads = gradient(() -> loss, params(corner_net))
        Optimise.update!(corner_optimizer, params(corner_net), grads)
        # retrieve the sorted states, actions, rewards
        # i corresponds to the ith session, 1 corresponds to the states of the ith session
        # this is initally size (4n-4+1, n^2) but needs to be (1, 4n-4+1, n^2)

        #####
        # TODO
        # currently just resetting super_states, need to stack them
        # possible error here, fix here or above.
        super_states = Array{Int}(undef, 0, 4*n - 4 + 1, n^2)
        super_actions = Array{Int}(undef, 0, 4*n - 4 + 1)
        for i in 1:length(super_sessions)
            # just use super_states_reshaped?
            super_states_reshaped = reshape(super_sessions[i][1], (1, size(super_sessions[i][1])...))
            super_states = cat(super_states, super_states_reshaped; dims=1)

            super_actions_reshaped = reshape(super_sessions[i][2], (1, size(super_sessions[i][2])...))
            super_actions = cat(super_actions, super_actions_reshaped; dims=1)
        end
        super_rewards = [super_sessions[i][3] for i in 1:length(super_sessions)]

        mean_best_reward = mean(super_rewards)

        println("\n$i. Best individuals: ", super_rewards)
        # Uncomment the line below to print out the mean best reward
        println("Mean best reward: $mean_best_reward")

        # Make a new folder if 'Data' folder does not exist
        if !isdir("Data_two_agent")
            mkdir("Data_two_agent")
        end

        max_index = argmax(super_rewards)
        #max_index = 1

        if super_rewards[max_index] > cur_best_reward
            cur_best_reward = super_rewards[max_index]
            cur_best_board = super_states[max_index, 4*n-4+1,:] # best board as vector
            cur_best_game = super_states[max_index,:,:]

            best_states_set = Set()
            push!(best_states_set, string(cur_best_board))
            
            # add to file
            open(joinpath("Data_two_agent", filename * "_best_board_timeline.txt"), "a") do f
                write(f, board_to_string(cur_best_board, n), '\n')
            end
            open(joinpath("Data_two_agent", filename * "_best_reward_timeline.txt"), "a") do f
                write(f, string(cur_best_reward), '\n')
            end
            if cur_best_reward == 424
                open(joinpath("Data_two_agent", filename * "_best_reward_timeline.txt"), "a") do f
                    write(f, "GENERATION $i", '\n')
                end
            end
        end
        
        if super_rewards[max_index] == cur_best_reward
            cur_best_board = super_states[max_index, 4*n-4+1,:] # best board as vector
            if !in(string(cur_best_board), best_states_set)
                push!(best_states_set, string(cur_best_board))

                # add to file
                open(joinpath("Data_two_agent", filename * "_best_board_timeline.txt"), "a") do f
                    write(f, board_to_string(cur_best_board, n), '\n')
                end
                open(joinpath("Data_two_agent", filename * "_best_reward_timeline.txt"), "a") do f
                    write(f, string(cur_best_reward), '\n')
                end
            end
    
        end
        
    end
        return net, corner_net, cur_best_game
end

train (generic function with 1 method)

# Training

In [32]:
#n_sessions = 200
n = 10
filename = "10x10"
best_net, best_corner_net, best_game = train(n, filename)


 GENERATION 1

1. Best individuals: [100, 96, 96, 66, 63, 60, 52, 52, 44, 40, 36, 36, 36, 36, 34, 32, 24, 24, 24, 20]
Mean best reward: 48.55

 GENERATION 2

2. Best individuals: [86, 70, 68, 63, 52, 50, 48, 48, 46, 44, 36, 36, 32, 28, 28, 24, 21, 20, 20, 20]
Mean best reward: 42.0

 GENERATION 3

3. Best individuals: [87, 80, 78, 74, 72, 72, 68, 66, 56, 56, 54, 54, 52, 40, 40, 40, 37, 36, 36, 36]
Mean best reward: 56.7

 GENERATION 4

4. Best individuals: [120, 106, 105, 95, 84, 81, 60, 52, 52, 52, 48, 48, 44, 40, 30, 28, 27, 24, 24, 24]
Mean best reward: 57.2

 GENERATION 5

5. Best individuals: [88, 80, 76, 64, 60, 48, 42, 40, 36, 36, 32, 30, 30, 28, 24, 24, 24, 24, 24, 24]
Mean best reward: 41.7

 GENERATION 6

6. Best individuals: [198, 94, 72, 60, 60, 51, 48, 44, 38, 36, 32, 30, 26, 24, 24, 24, 24, 24, 24, 24]
Mean best reward: 47.85

 GENERATION 7

7. Best individuals: [89, 80, 72, 58, 56, 52, 51, 50, 48, 44, 40, 36, 34, 32, 28, 26, 24, 24, 24, 24]
Mean best reward: 44.6

 GENE

LoadError: InterruptException: